In [1]:
# Import Libraries:
import datetime
import os
import re
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Setting up directory:
path = os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Raw Data")
path_ = os.getcwd()
path_

'C:\\ABI\\Q2_Competitive\\Ad_Spend\\Raw Data'

In [3]:
# Obtaining the files:
files = os.listdir(path_)
print("The number of markets",len(files))
files

The number of markets 23


['Argentina_Q2_2023 Competitive Request_Group I.xlsx',
 'Bolivia_Q2_2023 Competitive Request_Group II.xlsx',
 'Brazil_Q2_2023 Competitive Request_Group I.xlsx',
 'Canada_Q2_2023 Competitive Request _Group I - New.xlsx',
 'Chile_Q2_2023 Competitive Request_Group II.xlsx',
 'China_Q2_2023 Competitive Request_Group I.xlsx',
 'Colombia_Q2_2023 ABI Global Competitive_Group I - New.xlsx',
 'DR_Q2_2023 Competitive Request_Group II.xlsx',
 'Ecuador_Q2_2023 Competitive Request_Group II.xlsx',
 'El Salvador_Q2_2023 Competitive Request_Group II.xlsx',
 'Guatemala_Q2_2023 Competitive Request_Group II.xlsx',
 'Hondurus_Q2_2023 Competitive Request_group II.xlsx',
 'India_Q2_2023 Competitive Request_Group I.xlsx',
 'Japan_Q2_2023 Competitive Request_Group II.xlsx',
 'Mexico_Q2 _2023 Competitive Request_Group I New.xlsx',
 'Panama_Q2_2023 Competitive Request_Group II.xlsx',
 'Paraguay_Q2_2023 Competitive Request_Group II.xlsx',
 'Peru_Q2_2023 Competitive Request_Group II.xlsx',
 'South Africa_Q2_2023 

In [4]:
## Initialize Empty dataframe:
df = pd.DataFrame()
for f in files:
    data = pd.read_excel(f)
    df = df.append(data)

In [5]:
df.head()

,Country,Data Source,Advertiser (parent company),Brand,Category,Media type,Year,Month,Spend (Local Currency),Exchange Rate,Spend (US Currency)
0,ARGENTINA,IBOPE - AdMedia,PEPSI COLA,GATORADE.,ENERGY DRINK,TV Cable,2023,6,3181.423507,253.93,12.528742
1,ARGENTINA,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,2268.824968,253.93,8.934844
2,ARGENTINA,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,4537.649935,253.93,17.869688
3,ARGENTINA,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,5834.121345,253.93,22.975313
4,ARGENTINA,IBOPE - AdMedia,AGUAS DANONE,VILLAVICENCIO,AGUAS MINERALES Y MINERALIZADA,TV Cable,2023,2,2106.766041,253.93,8.296641


In [6]:
df.shape

(471079, 11)

In [7]:
df.columns

Index(['Country', 'Data Source', 'Advertiser (parent company)', 'Brand',
       'Category', 'Media type', 'Year', 'Month', 'Spend (Local Currency)',
       'Exchange Rate', 'Spend (US Currency)'],
      dtype='object')

In [8]:
#os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Processed Data")
#print(os.getcwd())
#df.to_excel("test output.xlsx", index=False)

In [9]:
# Renameing features: for easy indexing in python
new_columns = ['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency']
df.columns = new_columns
df.columns

Index(['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency'],
      dtype='object')

In [10]:
## Making the String in upper case:
for col in df.columns[:-7]:
    #print(col)
    df[col] = df[col].str.upper()

### Available Markets

In [11]:
df['Country'].unique()

array(['ARGENTINA', 'BOLIVIA', 'BRAZIL', 'CANADA', 'CHILE', 'CHINA',
       'COLOMBIA', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EL SALVADOR',
       'GUATEMALA', 'HONDURAS', 'INDIA', 'JAPAN', 'MEXICO', 'PANAMA',
       'PARAGUAY', 'PERU', 'SOUTH AFRICA', 'SOUTH KOREA', 'URUGUAY', 'US',
       'VIETNAM'], dtype=object)

In [12]:
## Creating requried columns with null values
df["Category_Standardized"] = np.nan
df["Advertiser_parent_standarized"] = np.nan
df["Brand_standarization"] = np.nan
df['Parent_competitor'] = np.nan

df["Top_Brand_Competitors_and_ABI"] = np.nan
df["Focused_Brand_Competitors"] = np.nan

df['Top10_Brands_by_Market'] = np.nan

df["Media_Summary"] = np.nan
df["Media_type_standarized"] = np.nan
df["Month_number"] = np.nan
df['Month_standarization'] = np.nan

df["Focused_Global_Brand"] = np.nan
df["Global_Parent"] = np.nan
df["ABI_Parent_SOS"] = np.nan
df["Heineken_Brand_SOS"] = np.nan
df["Amstel_Brand_SOS"] = np.nan
df["Digital_Traditional"] = np.nan
df["Heineken_Parent_SOS"] = np.nan
df["Brand_Rank_by_Country"] = np.nan
df["Market_Group"] = np.nan

### Category (standardized)

In [13]:
#df['Category'].unique()

In [14]:
# Drink Categorization:
beer = ["MALTAS", "BEER", "BEER - OTHER", "CERVEZA", "CERVEZAS", "CERVEZAS ABI", 
        "CERVEZAS ARTESANALES", "CERVEZAS CCC", "CERVEZAS RESTANTES", "EVENTS DRINKS", "INST. BAVARIA", 'BEER RANGES',
       'Light & Low Alcohol Beer & Ale', 'Regular Beer & Ale', 'Cerveza']

beyond_beer = ["SELTZER", 'beyond Beer', "COOLERS, READY TO DRINK"]

non_alcoholic_beer = ['Non Alc']

other_alcohol = ["ENERGY DRINK", "COLAS", "E-COMMERCE", "FERNETS", "GIN", "INSTITUC BEBIDAS ALCOHOLICAS", 
                 "JUGOS NATURALES", "LIMON DIET", "LINEA SABORES", "MANZANA", "NARANJA", "SIDRAS"," VINOS ALTA GAMA", 
                 "VINOS BAJA GAMA", "VINOS ESPUMANTES", "VINOS MEDIA GAMA", "Liqueur", "LIQUEURS"," Non-Beer", 
                 "OTHER", "Other Alcoholic Drinks", "OTHER LIQUORS", "OTROS LICORES", "RON", "RTD ALCOHOLIC BEVERAGES",
                 "RTD MIXES", "RUM", "Soda", "SODAS", "SPIRITS", "SPIRITS - OTHER", "UVA", "VINO", "VINOS", 
                 "VINOS FRIZZANTE", "VODKA", "WHISKY", "WHISKYS", "WINE", "WINE - OTHER", "WINE/SPARKLING WINE",
                 "AGUAS MINERALES Y MINERALIZADA", "AGUAS SABORIZADAS CON GAS", "AGUAS SABORIZADAS LINEA",
                 "AGUAS SABORIZADAS SIN GAS", "ALCOHOLIC BEVERAGES", "CAMPAÑAS DE CONSUMO ALCOHOLIC", "CHAMPAGNE",
                 "Destilled Spirit", "INSTITUC BEBIDAS SIN ALCOHOL", "LICORES", "LIMA LIMON, CITRUS",
                 "LINEA BEBIDAS ALCOHOLICAS", "LINEA BEBIDAS SIN ALCOHOL", "APERITIV SIN ALCOHOL Y VEGETAL",
                 "BEBIDAS ALCOHOLICAS PREPARADAS", "BEBIDAS ALCOHOLICAS SABORIZADAS", "DISTRIBUID BEBIDAS ALCOHOLICAS",
                 "FLAVORED ALCOHOLIC BEVERAGES", "GRAPAS", "IND. LICORERA", "AGUARDIENTE", "ALCOPOPS", "APERITIVOS",
                 "BEBIDAS DE SOJA", "CIDER", "CIDER - OTHER", "COCTELES", "COLAS DIET", "EMBOTELLADORAS", "FRUTAS",
                 "JUGOS EN POLVO PARA PREPARAR", 'VINOS ALTA GAMA', 'NON-BEER', 'LIQUEUR', 'DESTILLED SPIRIT', 
                 'OTHER ALCOHOLIC DRINKS', 'SODA', 'Cider', 'Wine', 'Alcoholic Beverages: Comb Copy & NEC', 'Non-Beer']

df["Category_Standardized"] = df['Category'].copy()
df["Category_Standardized"] = df["Category_Standardized"].replace(beer, "Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(beyond_beer, "Beyond Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(other_alcohol, "Other Alcohol")
df["Category_Standardized"] = df["Category_Standardized"].replace(non_alcoholic_beer, "Non Alcoholic Beer")
df['Category_Standardized'].unique()

array(['Other Alcohol', 'Beer', 'Beyond Beer', 'Non Alcoholic Beer'],
      dtype=object)

### Advertiser (standardized)

In [15]:
df["Advertiser_parent_standarized"] = df["Advertiser_parent_company"].copy()

In [16]:
# Codes needs to be optimized using map function and using dictionary


## Argentina:
df.loc[df['Advertiser_parent_standarized'] == "CCU SA", 'Advertiser_parent_standarized'] = "CCU SA (Heineken Parent)"
df.loc[df['Advertiser_parent_standarized'] == "QUILMES", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GOOSE ISLAND BEER COMPANY", 'Advertiser_parent_standarized'] = "ABI"

#Bolivia
df.loc[df['Advertiser_parent_standarized'] == "ABINVEB", 'Advertiser_parent_standarized'] = "ABI"


##Chile
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA CHILE", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CCU", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"
df.loc[df['Advertiser_parent_standarized'] == "PATAGONIA (CERVEZA)", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MOLSON COORS CHILE", 'Advertiser_parent_standarized'] = "MOLSON COORS"

##China
df.loc[df['Advertiser_parent_standarized'] == "Carlsberg Group".upper(), 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "CR Beer".upper(), 'Advertiser_parent_standarized'] = "CR BEER (Heineken Parent)"


##Colombia
# In 2005, Bavaria Brewery became a subsidiary of SABMiller. SABMiller is subsidiary of ABI
df.loc[df['Advertiser_parent_standarized'] == "BAVARIA", 'Advertiser_parent_standarized'] = "ABI" ## QURIED the market
df.loc[df['Advertiser_parent_standarized'] == "ABINBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CCC", 'Advertiser_parent_standarized'] = "CCC (Heineken Parent)"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN INT", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCA INT", 'Advertiser_parent_standarized'] = "CCA"


#'DOMINICAN REPUBLIC':
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA NACIONAL DOMINICANA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "UNITED BRANDS", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCU CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"
df.loc[df['Advertiser_parent_standarized'] == "BUDWEISER CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CLUB COLOMBIA CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "QUILMES CABLE PANREGIONALL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MIKES HARD CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "AGUILA CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"



#'ECUADOR'
df.loc[df['Advertiser_parent_standarized'] == "CIA. DE CERVEZAS NACIONALES", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "STELLA ARTOIS", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "Heineken", 'Advertiser_parent_standarized'] = "HEINEKEN"

#'EL SALVADOR'
df.loc[df['Advertiser_parent_standarized'] == "INDUSTRIAS LA CONSTANCIA SA DE CV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "DISAL", 'Advertiser_parent_standarized'] = "HEINEKEN"

#'GUATEMALA'
df.loc[df['Advertiser_parent_standarized'] == "AMBEV CENTROAMERICA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GRUPO MODELO CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "MIKES HARD LEMONADE CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA CENTRO AMERICANA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CCU CABLE PANREGIONAL", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"

#'HONDURAS'
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA HONDUREÑA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "DISTRIBUIDORA ISTMANIA", 'Advertiser_parent_standarized'] = "HEINEKEN"

#'INDIA'
df.loc[df['Advertiser_parent_standarized'] == "Pernod Ricard india private limited", 'Advertiser_parent_standarized'] = "Pernod Ricard".upper()
df.loc[df['Advertiser_parent_standarized'] == "bacardi martini india", 'Advertiser_parent_standarized'] = "Bacardi".upper()
df.loc[df['Advertiser_parent_standarized'] == "CROWN BEERS INDIA LTD", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "Carlsberg Beer".upper(), 'Advertiser_parent_standarized'] = "Carlsberg".upper()
df.loc[df['Advertiser_parent_standarized'] == "united spirits limited", 'Advertiser_parent_standarized'] = "United Spirits".upper()
df.loc[df['Advertiser_parent_standarized'] == "Heineken N.V.", 'Advertiser_parent_standarized'] = "HEINEKEN"

#'JAPAN' - na
#'MEXICO' -na

#'PARAGUAY'
df.loc[df['Advertiser_parent_standarized'] == "Bebidas del Paraguay", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "Distribuidora Gloria", 'Advertiser_parent_standarized'] = "MOLSON COORS"



#'PANAMA' ## is FEDURO should be Molson Coors - Ask Bob 
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA PANAMA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "DIST COMERCIAL GROUP", 'Advertiser_parent_standarized'] = "ABI"

#'PERU'
df.loc[df['Advertiser_parent_standarized'] == "UCP BACKUS Y JOHNSTON S.A.A.", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "LIMBA SERVICES SAC", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "DESTILERIA PERUANA", 'Advertiser_parent_standarized'] = "BACARDI"

#'SOUTH AFRICA'
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER-BUSCH INBEV", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "SAB MILLER", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN INTERNATIONAL", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN SOUTH AFRICA", 'Advertiser_parent_standarized'] = "HEINEKEN"
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA MODELO", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "BELGIUM BEER COMPANY", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "ANHEUSER BUSCH INC", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CORONA CHASING SUNSETS", 'Advertiser_parent_standarized'] = "ABI"

#'SOUTH KOREA'
df.loc[df['Advertiser_parent_standarized'] == "ABInBev", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "GOLDEN BLUE", 'Advertiser_parent_standarized'] = "GOLDEN BLUE (MOLSON COORS)"

#'URUGUAY'
df.loc[df['Advertiser_parent_standarized'] == "FNC", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "FABRICAS NACIONALES DE CERVEZA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "CORONA", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "PILSEN", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == "COMPANIA CERVECERIAS UNIDAS", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"
df.loc[df['Advertiser_parent_standarized'] == "CCU", 'Advertiser_parent_standarized'] = "CCU (Heineken Parent)"

#'VIETNAM'
df.loc[df['Advertiser_parent_standarized'] == "CARLSBERG VIETNAM", 'Advertiser_parent_standarized'] = "CARLSBERG"
df.loc[df['Advertiser_parent_standarized'] == "VBL", 'Advertiser_parent_standarized'] = "VBL (Heineken Parent)"

# USA
df.loc[df['Advertiser_parent_standarized'] == "Anheuser-Busch InBev", 'Advertiser_parent_standarized'] = "ABI"
df.loc[df['Advertiser_parent_standarized'] == 'MOLSON COORS BREWING CO', 'Advertiser_parent_standarized'] = "MOLSON COORS"
df.loc[df['Advertiser_parent_standarized'] == "HEINEKEN NV", 'Advertiser_parent_standarized'] = "HEINEKEN"

#Bolivia
df.loc[df['Advertiser_parent_standarized'] == "CERVECERIA BOLIVIANA NACIONAL S.A.", 'Advertiser_parent_standarized'] = "ABI"

#df["Advertiser_parent_standarized"].unique()

### Brand (standardized) 

In [17]:
df["Brand_standarization"] = df["Brand"].copy() # Copy the brnad names to a new column to strd

df["Brand_standarization"] = df["Brand_standarization"].str.strip()
df["Brand_standarization"] = df["Brand_standarization"].str.replace('SPONSORSHIP', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('CORPORATE', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('COMPETITION', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('RUGBY', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('REG', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('BEER'.upper(), '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace('CERVEZA', '')
df["Brand_standarization"] = df["Brand_standarization"].str.replace("CERVECERIA", "")
df["Brand_standarization"] = df["Brand_standarization"].str.replace("(", "")
df["Brand_standarization"] = df["Brand_standarization"].str.replace(")", "")

df["Brand_standarization"] = df["Brand_standarization"].str.strip()

df.loc[df['Brand_standarization'] == "HEINEKEN 0 0", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN O.O LAGER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN 0 0 A1PADEL PAYSPACE CAPE TOWN MASTER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN O.O LAGER", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN 00", 'Brand_standarization'] = "HEINEKEN 0.0"
df.loc[df['Brand_standarization'] == "HEINEKEN MEXICO", 'Brand_standarization'] = "HEINEKEN"
df.loc[df['Brand_standarization'] == "HEINEKEN ()", 'Brand_standarization'] = "HEINEKEN"



df.loc[df['Brand_standarization'] == 'UNITED BRANDS', 'Brand_standarization'] = "HEINEKEN" #Dominican Republic

df.loc[df['Brand_standarization'] == "GC/AMSTEL MALTA", 'Brand_standarization'] = "AMSTEL MALTA"




C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/66319622.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Brand_standarization"] = df["Brand_standarization"].str.replace("(", "")
C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/66319622.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Brand_standarization"] = df["Brand_standarization"].str.replace(")", "")


In [18]:
## Other Brands including ABI


df.loc[df['Brand_standarization'] == "BRUGAL ()", 'Brand_standarization'] = "BRUGAL"
df.loc[df['Brand_standarization'] == "CORONA.", 'Brand_standarization'] = "CORONA" # 'ARGENTINA'
df.loc[df['Brand_standarization'] == 'PACEÑA', 'Brand_standarization'] = "PACENA" # Bolivia
df.loc[df['Brand_standarization'] == 'AUSTRAL (CERVEZA)', 'Brand_standarization'] = "AUSTRAL" #'CHILE'
df.loc[df['Brand_standarization'] == 'CRISTAL (CERVEZA)', 'Brand_standarization'] = "CRISTAL"
df.loc[df['Brand_standarization'] == 'PATAGONIA CERVEZA', 'Brand_standarization'] = "PATAGONIA"
df.loc[df['Brand_standarization'] == 'PATAGONIA (CERVEZA)', 'Brand_standarization'] = "PATAGONIA"
df.loc[df['Brand_standarization'] == 'SOL (CERVEZA)', 'Brand_standarization'] = "SOL"
df.loc[df['Brand_standarization'] == 'ABINBEV INSTITUCIONAL', 'Brand_standarization'] = "ABI"
df.loc[df['Brand_standarization'] == 'CORONA (CERVEZA)', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CUSQUEÑA', 'Brand_standarization'] = "CUSQUENA"
df.loc[df['Brand_standarization'] == "Snow Others", 'Brand_standarization'] = "SNOW"  #'CHINA'
df.loc[df['Brand_standarization'] == "Others", 'Brand_standarization'] = "All Others"
df.loc[df['Brand_standarization'] == "CORONITA", 'Brand_standarization'] = "CORONA"  #'COLOMBIA'
df.loc[df['Brand_standarization'] == "COSTEÑA", 'Brand_standarization'] = "COSTENA"
df.loc[df['Brand_standarization'] == 'CERVECERIA NACIONAL DOMINICANA', 'Brand_standarization'] = "CERVECERIA" #'Dom Rep'
df.loc[df['Brand_standarization'] == 'CIA. CERVEZAS NACIONALES', 'Brand_standarization'] = "ABI"  #'ECUADOR'
df.loc[df['Brand_standarization'] == 'CASTLE LITE BEER', 'Brand_standarization'] = "CASTLE LITE" #'SOUTH AFRICA'
df.loc[df['Brand_standarization'] == 'BRUTAL FRUIT SPRITZER', 'Brand_standarization'] = "BRUTAL FRUIT"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER COMPETITION', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CARLING BLACK LABEL BEER SOCCER SPONSORSHIP', 'Brand_standarization'] = "CARLING BLACK LABEL"
df.loc[df['Brand_standarization'] == 'CASTLE LITE BEER COMPETITION', 'Brand_standarization'] = "CASTLE LITE"
df.loc[df['Brand_standarization'] == 'FLYING FISH CHILL BEER', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS COMPETITION', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH PREMIUM FLAVOURED BEER', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH FLAVOUR CHILLAS COMPETITION', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH PREMIUM FLAVOURED BEER COMP', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH SELTZER', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'FLYING FISH SELTZER PROMOTION', 'Brand_standarization'] = "FLYING FISH"
df.loc[df['Brand_standarization'] == 'HUNTERS CIDER COMPETITION', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS CIDER RANGE', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS DRY CIDER', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'HUNTERS REFRESH THE RULES OF IGRAND', 'Brand_standarization'] = "HUNTERS"
df.loc[df['Brand_standarization'] == 'SAVANNA CIDER MUSIC SPONSORSHIP', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA CIDER RANGE', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA COMICS AWARDS', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA DRY CIDER', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'SAVANNA DRY CIDER COMEDY SPONSORSHIP', 'Brand_standarization'] = "SAVANNA"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY COMPETITION', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'THREE SHIPS WHISKY PROMOTION', 'Brand_standarization'] = "THREE SHIPS"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER BLACK LABEL', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER FLAVOURCODE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER GOLD LABEL RESERVE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'JOHNNIE WALKER WHISKY RANGE', 'Brand_standarization'] = "JOHNNIE WALKER"
df.loc[df['Brand_standarization'] == 'CORONA SUNSET FESTIVAL', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CORONA OPEN JBAY', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'BLACK CROWN PREMIUM GIN & TONIC', 'Brand_standarization'] = "BLACK CROWN"
df.loc[df['Brand_standarization'] == 'STELLA ARTOIS BEER', 'Brand_standarization'] = "STELLA ARTOIS"
df.loc[df['Brand_standarization'] == 'STELLA ARTOIS PLAYERS CHAMPIONSHIP', 'Brand_standarization'] = "STELLA ARTOIS"
df.loc[df['Brand_standarization'] == 'BUDWEISER BEER', 'Brand_standarization'] = "BUDWEISER"
df.loc[df['Brand_standarization'] == 'CORONA CHASING SUNSETS', 'Brand_standarization'] = "CORONA"
df.loc[df['Brand_standarization'] == 'CASTLE MILK STOUT MEDLEY PROMOTION', 'Brand_standarization'] = "CASTLE MILK STOUT"
df.loc[df['Brand_standarization'] == 'Bud Light Corporate'.upper(), 'Brand_standarization'] = "Bud Light".upper() #USA 
df.loc[df['Brand_standarization'] == 'Bud Light Seltzer Corporate'.upper(), 'Brand_standarization'] = "Bud Light Seltzer".upper()
df.loc[df['Brand_standarization'] == 'Budweiser Corporate'.upper(), 'Brand_standarization'] = "Budweiser".upper()
df.loc[df['Brand_standarization'] == 'Corona Extra'.upper(), 'Brand_standarization'] = "Corona".upper()
df.loc[df['Brand_standarization'] == 'Corona Extra Corporate'.upper(), 'Brand_standarization'] = "Corona".upper()
df.loc[df['Brand_standarization'] == 'Dos Equis Corporate'.upper(), 'Brand_standarization'] = "Dos Equis".upper()
df.loc[df['Brand_standarization'] == 'Michelob Ultra Corporate'.upper(), 'Brand_standarization'] = "Michelob Ultra".upper()
df.loc[df['Brand_standarization'] == 'Miller Lite Corporate'.upper(), 'Brand_standarization'] = "Miller Lite".upper()
df.loc[df['Brand_standarization'] == 'Modelo Corporate'.upper(), 'Brand_standarization'] = "Modelo Especial".upper()
df.loc[df['Brand_standarization'] == 'Anheuser-Busch'.upper(), 'Brand_standarization'] = "ABI"
df.loc[df['Brand_standarization'] == 'Anheuser-Busch Corporate'.upper(), 'Brand_standarization'] = "ABI"                                         
df.loc[df['Brand_standarization'] == 'Budweiser Sponsorship'.upper(), 'Brand_standarization'] =  "Budweiser".upper()                                          
df.loc[df['Brand_standarization'] == 'Bud Light Sponsorship'.upper(), 'Brand_standarization'] =  "Bud Light".upper()              
df.loc[df['Brand_standarization'] == 'Cerveza Victoria'.upper(), 'Brand_standarization'] =  "Victoria".upper() 
                                           

In [19]:
#Code Test
#brands = df['Brand_standarization'].value_counts().to_frame()
#brands = brands.reset_index()

### Focused Brand Competitors

In [20]:
## Focus brand for the market:  It woudl be constant throughout the year

focus_brands = ['HEINEKEN', "HEINEKEN SILVER", "HEINEKEN 0.0", "HEINEKEN LAGER", "AMSTEL", "AMSTEL ULTRA", 
               "AMSTEL ULTRA SELTZER", "AMSTEL LAGER", "AMSTEL MALTA", "AMSTEL LIGHT"]

df["Focused_Brand_Competitors"] = df["Brand_standarization"].copy()

# Assign most values in the 'Top_Brand_Competitors_and_ABI' column as 'Other' except for specific values
df['Focused_Brand_Competitors'] = np.where(df['Focused_Brand_Competitors'].isin(focus_brands), 
                                              df['Focused_Brand_Competitors'], 'All Others')
df['Focused_Brand_Competitors'].unique()

array(['All Others', 'HEINEKEN', 'AMSTEL', 'AMSTEL ULTRA', 'HEINEKEN 0.0',
       'HEINEKEN SILVER', 'AMSTEL ULTRA SELTZER', 'AMSTEL LAGER',
       'AMSTEL MALTA', 'HEINEKEN LAGER', 'AMSTEL LIGHT'], dtype=object)

### Media Summary

In [21]:
#df.Media_type.unique()

In [23]:
# Standarize Media Summary column
df["Media_Summary"] = df["Media_type"].copy() 

tv = ['Cable TV', 'PAY TV', 'Span Lang Net TV', 'TVPagada', 'CABLE', 'LOCAL TV', 'Network TV', 'OPEN TV', 'OTV', 
      'PAID TV (CABLE)', 'PTV', 'Spot TV', 'TELEVISION', 'TV', 'tv', 'TV ABIERTA', 'TV Abierta', 'TV ABIERTA ', 'TV CABLE',
      'TV Capital', 'TV Interior', 'TV PAGADA', 'TV Cable', 'Syndication']
radio = ['Natl Spot Radio', 'Radio AM', 'Local Radio', 'Network Radio', 'RADIO', 'Radio', 'Radio FM', 'Radios', 'RD']
print_ = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'PRESS', 'REVISTAS', 'B-to-B Magazines', 'DIARIOS', 
          'Diarios Interior', 'Hispanic Newspapers', 'Local Magazines', 'Local Media', 'MAGAZINE', 'Magazine', 'Magazines',
          'METRO', 'Newspaper', 'NP', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT', 'Revista', 'Revistas', 'Suplemento']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública', 'Cinema', 'CINEMA', 'VP', 'Search']
digital = ['INTERNET', 'Internet', 'Internet - Display', 'Internet - Search', 'Digital', 'Programmatic', 
           'DIGITAL', 'META', 'Mobile App', 'Mobile Web', 'Mobile Web Video', 'Online Video', 'Paid Social', 'Social',
           'Youtube', 'AVOD']

df['Media_Summary'] = df['Media_Summary'].replace(tv, "TV")
df['Media_Summary'] = df['Media_Summary'].replace(radio, "Radio")
df['Media_Summary'] = df['Media_Summary'].replace(print_, "Print")
df['Media_Summary'] = df['Media_Summary'].replace(ooh, "OOH")
df['Media_Summary'] = df['Media_Summary'].replace(digital, "Digital")

df['Media_Summary'].unique()

array(['TV', 'Radio', 'Print', 'Digital', 'OOH'], dtype=object)

### Media Type (Standardized)

In [24]:
# Media Type (Standardized)
df["Media_type_standarized"] = df["Media_type"].copy() 

cable_tv = ['Cable TV', 'CABLE', 'PAID TV (CABLE)', 'TV CABLE', 'TV Cable']
open_tv = ['OPEN TV', 'OTV']
paid_tv = ['PAY TV', 'TVPagada', 'PTV', 'TV PAGADA']
free_tv = ['TV ABIERTA', 'TV Abierta', 'TV ABIERTA ']
spot_tv = [ 'Spot TV']
tv = ['Span Lang Net TV', 'AVOD', 'TELEVISION', 'TV', 'tv', 'TV Capital', 'TV Interior', 'LOCAL TV', 'Syndication']
am_radio = ['Radio AM']
fm_radio = [ 'Radio FM']
local_radio = [ 'Local Radio']
sports_radio = ['Natl Spot Radio']
network_radio = [ 'Network Radio']
radio = ['RADIO', 'Radio', 'Radios', 'RD']
cinema = ['Cinema', 'CINEMA']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública',  'VP']
print_ = ['PRESS', 'DIARIOS', 'Local Media', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT']
newspapers = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'Diarios Interior', 'Hispanic Newspapers',
              'METRO', 'Newspaper', 'NP', 'Suplemento']
magazines = ['REVISTAS', 'B-to-B Magazines', 'Local Magazines', 'MAGAZINE', 'Magazine', 'Magazines', 'Revista', 
             'Revistas']
digital = ['Digital', 'Programmatic', 'Syndication', 'DIGITAL', 'META', 'Mobile App', 'Mobile Web', 'Mobile Web Video', 
           'Online Video', 'Paid Social', 'Social', 'Youtube', 'INTERNET', 'Internet', 'Internet - Display', 
           'Internet - Search', 'AVOD', 'Search']

df['Media_type_standarized'] = df['Media_type_standarized'].replace(cable_tv, "Cable TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(open_tv, "Open TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(paid_tv, "Paid TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(free_tv, "Free TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(spot_tv, "Spot TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv, "TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(am_radio, "AM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(fm_radio, "FM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_radio, "Local Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(sports_radio, "Sports Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(network_radio, "Network Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(radio, "Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(cinema, "Cinema")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(print_, "Print")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(newspapers, "Newspapers")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(magazines, "Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(ooh, "OOH")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(digital, "Digital")

df['Media_type_standarized'].unique()

array(['Cable TV', 'FM Radio', 'TV', 'Newspapers', 'AM Radio',
       'Magazines', 'Digital', 'OOH', 'Print', 'Open TV', 'Paid TV',
       'Radio', 'Cinema', 'Free TV', 'Sports Radio', 'Local Radio',
       'Spot TV', 'Network TV', 'Network Radio'], dtype=object)

### Month (Number)

In [25]:
# Month (Number)
df["Month_number"] = df["Month"].copy()
df["Month_number"].unique()

array([6, 2, 4, 5, 1, 3, 'June', 'April', 'May', 'APR', 'FEB', 'JAN',
       'JUN', 'MAR', 'MAY', 'January', 'February', 'March', 'JUNIO',
       'ABRIL', 'MAYO', 'ABR', 'Enero', 'Febrero', 'Marzo', 'Abril',
       'Mayo', 'Junio', 'Julio', 'enero', 'febrero', 'marzo', 'abril',
       'mayo', 'junio', 'julio', 'Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul',
       'Ene', 'Abr', 'mar', 'APRIL', 'JUNE', 'FEBRUARY', 'MARCH',
       'JANUARY', Timestamp('2023-01-01 00:00:00'),
       Timestamp('2023-02-01 00:00:00'), Timestamp('2023-03-01 00:00:00'),
       Timestamp('2023-04-01 00:00:00'), Timestamp('2023-05-01 00:00:00'),
       Timestamp('2023-06-01 00:00:00'), 'JUn'], dtype=object)

In [27]:

df.loc[df['Month_number'] == "Jan", 'Month_number'] = 1
df.loc[df['Month_number'] == "JAN", 'Month_number'] = 1
df.loc[df['Month_number'] == "JANUARY", 'Month_number'] = 1
df.loc[df['Month_number'] == "January", 'Month_number'] = 1
df.loc[df['Month_number'] == "Ene", 'Month_number'] = 1
df.loc[df['Month_number'] == "enero", 'Month_number'] = 1
df.loc[df['Month_number'] == "Enero", 'Month_number'] = 1

df.loc[df['Month_number'] == "FEB", 'Month_number'] = 2
df.loc[df['Month_number'] == "Feb", 'Month_number'] = 2
df.loc[df['Month_number'] == "Febrero", 'Month_number'] = 2
df.loc[df['Month_number'] == "febrero", 'Month_number'] = 2
df.loc[df['Month_number'] == "FEBRUARY", 'Month_number'] = 2
df.loc[df['Month_number'] == "February", 'Month_number'] = 2

df.loc[df['Month_number'] == "MAR", 'Month_number'] = 3
df.loc[df['Month_number'] == "Mar", 'Month_number'] = 3
df.loc[df['Month_number'] == "mar", 'Month_number'] = 3
df.loc[df['Month_number'] == "MARCH", 'Month_number'] = 3
df.loc[df['Month_number'] == "March", 'Month_number'] = 3
df.loc[df['Month_number'] == "Marzo", 'Month_number'] = 3
df.loc[df['Month_number'] == "marzo", 'Month_number'] = 3

df.loc[df['Month_number'] == "ABR", 'Month_number'] = 4
df.loc[df['Month_number'] == "Abr", 'Month_number'] = 4
df.loc[df['Month_number'] == "Abril", 'Month_number'] = 4
df.loc[df['Month_number'] == "abril", 'Month_number'] = 4
df.loc[df['Month_number'] == "ABRIL", 'Month_number'] = 4
df.loc[df['Month_number'] == "Apr", 'Month_number'] = 4
df.loc[df['Month_number'] == "APR", 'Month_number'] = 4
df.loc[df['Month_number'] == "April", 'Month_number'] = 4
df.loc[df['Month_number'] == "APRIL", 'Month_number'] = 4

df.loc[df['Month_number'] == "MAY", 'Month_number'] = 5
df.loc[df['Month_number'] == "May", 'Month_number'] = 5
df.loc[df['Month_number'] == "Mayo", 'Month_number'] = 5
df.loc[df['Month_number'] == "mayo", 'Month_number'] = 5
df.loc[df['Month_number'] == "MAYO", 'Month_number'] = 5

df.loc[df['Month_number'] == "Jun", 'Month_number'] = 6
df.loc[df['Month_number'] == "June", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUNIO", 'Month_number'] = 6
df.loc[df['Month_number'] == "Junio", 'Month_number'] = 6
df.loc[df['Month_number'] == "junio", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUN", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUNE", 'Month_number'] = 6
df.loc[df['Month_number'] == "JUn", 'Month_number'] = 6

df.loc[df['Month_number'] == "Julio", 'Month_number'] = 7
df.loc[df['Month_number'] == "julio", 'Month_number'] = 7
df.loc[df['Month_number'] == "Jul", 'Month_number'] = 7

df.loc[df['Month_number'] == pd.Timestamp('2023-01-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-01-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-02-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-02-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-03-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-03-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-04-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-04-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-05-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-05-01 00:00:00').month
df.loc[df['Month_number'] == pd.Timestamp('2023-06-01 00:00:00'), 'Month_number'] = pd.to_datetime('2023-06-01 00:00:00').month
 
df["Month_number"].unique()

array([6, 2, 4, 5, 1, 3, 7], dtype=object)

### Month (Standardized)

In [29]:
df['Month_standarization'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month_number'].astype(str))

### Top Brand Competitors + ABI

In [27]:
## Null - Not requried


#### Creating Dataframes Market Wise

In [30]:
# Its requried to create number of filtered dataframes for each country to use 
# pivot table to find top competetor, top 10 markets

df_argentina = df[df['Country'] == 'ARGENTINA']
df_bolivia = df[df['Country'] == 'BOLIVIA']
df_brazil = df[df['Country'] == 'BRAZIL']
df_canada = df[df['Country'] == 'CANADA']
df_chile = df[df['Country'] == 'CHILE']
df_china = df[df['Country'] == 'CHINA']
df_colombia = df[df['Country'] == 'COLOMBIA']
df_dom_rep = df[df['Country'] == 'DOMINICAN REPUBLIC']
df_ecuador = df[df['Country'] == 'ECUADOR']
df_el_salvador = df[df['Country'] == 'EL SALVADOR']
df_guatemala = df[df['Country'] == 'GUATEMALA']
df_honduras = df[df['Country'] == 'HONDURAS']
df_india = df[df['Country'] == 'INDIA']
df_japan = df[df['Country'] == 'JAPAN']
df_mexico = df[df['Country'] == 'MEXICO']
df_panama = df[df['Country'] == 'PANAMA']
df_paraguay = df[df['Country'] == 'PARAGUAY']
df_peru = df[df['Country'] == 'PERU']
df_south_africa = df[df['Country'] == 'SOUTH AFRICA']
df_south_korea = df[df['Country'] == 'SOUTH KOREA']
df_uruguay = df[df['Country'] == 'URUGUAY']
df_us = df[df['Country'] == 'US']
df_vietnam = df[df['Country'] == 'VIETNAM']

markets = [df_argentina, df_bolivia, df_brazil, df_canada, df_chile, df_china, df_colombia, df_dom_rep, df_ecuador, 
           df_el_salvador, df_guatemala, df_honduras, df_india, df_japan, df_mexico, df_panama, df_paraguay, df_peru, 
           df_south_africa, df_south_korea, df_uruguay, df_us, df_vietnam]

print("The Number of markets for Competitive Analysis: {}\n".format(len(markets)))

for market in markets:
    print("The number of data points for country: {}\t\t:{}".format(market["Country"].unique(), market.shape[0]))

The Number of markets for Competitive Analysis: 23

The number of data points for country: ['ARGENTINA']		:175765
The number of data points for country: ['BOLIVIA']		:50
The number of data points for country: ['BRAZIL']		:445
The number of data points for country: ['CANADA']		:1772
The number of data points for country: ['CHILE']		:142
The number of data points for country: ['CHINA']		:273
The number of data points for country: ['COLOMBIA']		:376
The number of data points for country: ['DOMINICAN REPUBLIC']		:66398
The number of data points for country: ['ECUADOR']		:441
The number of data points for country: ['EL SALVADOR']		:5649
The number of data points for country: ['GUATEMALA']		:79229
The number of data points for country: ['HONDURAS']		:5277
The number of data points for country: ['INDIA']		:162
The number of data points for country: ['JAPAN']		:234
The number of data points for country: ['MEXICO']		:23358
The number of data points for country: ['PANAMA']		:185
The number of da

### Parent Competitors (Pivots)

In [34]:
## Excluding Other Alcohol category for Competitive analysis of Beer

## Operation steps : 
# 1. filerter datframe by excluding Other Alcohol category
# 2. Use groupby fuction by adv. parent with aggregation with percentage applying lambda fucntion
# 3. Sort values in descending
# 4. Spend_US_Currency column in pivot table would be in percentage

## Function to find top 5 parent competitor in a market in beer category given their ad spend market share in above 5%

def parent_competitors_std(df):
    competitors_df = df[(df['Category_Standardized'] 
                         != 'Other Alcohol')].groupby([ 'Advertiser_parent_standarized',
                                                       'Category_Standardized']).agg({'Spend_US_Currency': 
                                                                                      lambda x: 
                                                                                      round(x.sum()/df["Spend_US_Currency"].sum()*100,
                                                                                            2)}).sort_values('Spend_US_Currency', 
                                                                                                             ascending = False)
    competitors_df = competitors_df.reset_index()
    competitors_df = (competitors_df[(competitors_df["Spend_US_Currency"] >= 5) |
                                    (competitors_df["Advertiser_parent_standarized"].str.startswith('(HEINEKEN)|(AMSTEL)'))])
    
    exceptions = competitors_df["Advertiser_parent_standarized"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Parent_competitor'] = df['Advertiser_parent_standarized'].copy()
    df['Parent_competitor'] = np.where(df['Parent_competitor'].isin(exceptions), 
                                                   df['Parent_competitor'], "All Others")
    print("\n")
    return df['Parent_competitor'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), parent_competitors_std(market)))

C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/1579470207.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parent_competitor'] = df['Advertiser_parent_standarized'].copy()
C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/1579470207.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parent_competitor'] = np.where(df['Parent_competitor'].isin(exceptions),




['ARGENTINA']:['All Others' 'ABI' 'CCU SA (Heineken Parent)']


['BOLIVIA']:['All Others' 'ABI']


['BRAZIL']:['ABI' 'HEINEKEN' 'All Others' 'PETROPOLIS']


['CANADA']:['MOLSON' 'HEINEKEN' 'All Others' 'MARK ANTHONY BREWING' 'ABI']


['CHILE']:['ABI' 'All Others' 'CCU (Heineken Parent)']


['CHINA']:['ABI' 'All Others' 'CARLSBERG' 'CR BEER (Heineken Parent)' 'TSINGTAO'
 'YANJING' 'OTHERS']


['COLOMBIA']:['All Others' 'ABI' 'CCC (Heineken Parent)']


['DOMINICAN REPUBLIC']:['All Others' 'CERVECERIA CENTRO AMERICANA CABLE PANREGIONAL']


['ECUADOR']:['ABI' 'HEINEKEN' 'All Others']


['EL SALVADOR']:['CERVECERIA CENTROAMERICANA' 'ABI' 'CADEJO' 'HEINEKEN' 'All Others']


['GUATEMALA']:['HEINEKEN' 'All Others' 'ABI']


['HONDURAS']:['DISTRIBUIDORA SOLIS' 'ABI' 'CERVECERIA CENTROAMERICANA' 'All Others']


['INDIA']:['UNITED BREWERIES' 'CARLSBERG' 'All Others']


['JAPAN']:['KIRIN' 'SAPPORO' 'ASAHI' 'SUNTORY' 'All Others']


['MEXICO']:['ABI' 'HEINEKEN']


['PANAMA']:['ABI' 'HEINEKEN' 'FED

### Top 10 Brands by Market

In [35]:
def top_10_brands(df):
    top10_df = df[(df['Category_Standardized'] != 
                  'Other Alcohol')].groupby([ 'Brand_standarization',
                                             'Category_Standardized']).agg({'Spend_US_Currency': 
                                                                            "sum"}).sort_values('Spend_US_Currency',
                                                                                                ascending = False).head(10)
    top10_df = top10_df.reset_index()
        
    exceptions = top10_df["Brand_standarization"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Top10_Brands_by_Market'] = df['Brand_standarization'].copy()
    df['Top10_Brands_by_Market'] = np.where(df['Top10_Brands_by_Market'].isin(exceptions), 
                                                   df['Top10_Brands_by_Market'], "All Others")
    
  
    print("\n")
    return df['Top10_Brands_by_Market'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), top_10_brands(market)))

C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/1956251057.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Top10_Brands_by_Market'] = df['Brand_standarization'].copy()
C:\Users\amimaiti\AppData\Local\Temp/ipykernel_13976/1956251057.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Top10_Brands_by_Market'] = np.where(df['Top10_Brands_by_Market'].isin(exceptions),




['ARGENTINA']:['All Others' 'HEINEKEN' 'IMPERIAL' 'BRAHMA' 'ANDES' 'SANTA FE' 'QUILMES'
 'AMSTEL' 'STELLA ARTOIS' 'NORTE' 'DR LEMON']


['BOLIVIA']:['BURGUESA' 'HUARI' 'PACENA' 'GOLDEN' 'POTOSINA']


['BRAZIL']:['All Others' 'BRAHMA' 'BUDWEISER' 'BUDWEISER ZERO' 'CORONA' 'SKOL'
 'SPATEN' 'AMSTEL' 'HEINEKEN' 'HEINEKEN 0.0' 'ITAIPAVA']


['CANADA']:['All Others' 'COORS LIGHT' 'MOLSON PARTNERSHIP' 'MOLSON ULTRA LOW CARB'
 'HEINEKEN' 'WHITE CLAW' 'BLACK FLY COOLERS' 'BUD LIGHT' 'BUDWEISER'
 'MICHELOB ULTRA' 'STELLA ARTOIS']


['CHILE']:['All Others' 'COORS' 'CRISTAL' 'ESCUDO' 'HEINEKEN' 'IMPERIAL' 'PATAGONIA'
 'ROYAL GUARD' 'SOL' 'STONES' 'BECKER']


['CHINA']:['BUDWEISER' 'All Others' 'HARBIN' 'BLUE GIRL' 'OTHERS' 'TUBORG' 'WUSU'
 'HEINEKEN' 'SNOW' 'TSINGTAO' 'YANJING']


['COLOMBIA']:['All Others' 'AGUILA' 'BAVARIA' 'BUDWEISER' 'CLUB COLOMBIA' 'CORONA'
 'HEINEKEN' 'POKER' 'STELLA ARTOIS' 'ANDINA' 'AMSTEL']


['DOMINICAN REPUBLIC']:['REPUBLICA' 'All Others' 'CLUB COLOMBIA' 'BUDWEISER' '

### Merging all the files back to main datafram

In [36]:
df_ = pd.DataFrame()
for market in markets:
    df_ = df_.append(market)

In [37]:
df_.columns

Index(['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency', 'Category_Standardized',
       'Advertiser_parent_standarized', 'Brand_standarization',
       'Parent_competitor', 'Top_Brand_Competitors_and_ABI',
       'Focused_Brand_Competitors', 'Top10_Brands_by_Market', 'Media_Summary',
       'Media_type_standarized', 'Month_number', 'Month_standarization',
       'Focused_Global_Brand', 'Global_Parent', 'ABI_Parent_SOS',
       'Heineken_Brand_SOS', 'Amstel_Brand_SOS', 'Digital_Traditional',
       'Heineken_Parent_SOS', 'Brand_Rank_by_Country', 'Market_Group'],
      dtype='object')

**** DO NOT EXECUTE CODE AFTER THIS LINE***

### CODE TESTING

In [38]:
import time
start_time = time.time()

os.chdir(r"C:\ABI\Q2_Competitive\Ad_Spend\Processed Data")
print(os.getcwd())
df_.to_excel("QII-Competitive Ad Spend.xlsx", index=False)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")
print(f"Elapsed time: {elapsed_time/60} minutes")

C:\ABI\Q2_Competitive\Ad_Spend\Processed Data
Elapsed time: 425.5279541015625 seconds
Elapsed time: 7.092132568359375 minutes
